# Install Transformers

In [173]:
def print_encoded(encoded_inputs):
    for ts in encoded_inputs:
        print(ts, encoded_inputs[ts])
        
        
from huggingface_hub import notebook_login

notebook_login()

In [2]:
%%bash
git config --global credential.helper store

# 1. Training

In [3]:
!nvidia-smi

Mon Mar  7 07:27:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   33C    P0    36W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   34C    P0    36W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## 1.1 Prepare-Datasets

In [4]:
import transformers
print(transformers.__version__)

4.17.0


In [5]:
model_checkpoint = "Helsinki-NLP/opus-mt-th-en"

In [6]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset

raw_datasets = load_dataset("scb_mt_enth_2020", name="then")
metric = load_metric("sacrebleu")

Reusing dataset scb_mt_enth2020 (/root/.cache/huggingface/datasets/scb_mt_enth2020/then/1.0.0/3bb5a43042a84446b254e215a8b1604b454ad56ada3370309a6f6a3b480c86a8)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 801402
    })
    validation: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 100173
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 100177
    })
})

In [8]:
raw_datasets['train'][0]['translation']

{'en': 'FAR LEFT: Indonesian National Police Chief Tito Karnavian, from left, Philippine National Police Chief Ronald Dela Rosa and Royal Malaysian Police Inspector General Khalid Abu Bakar link arms before the Trilateral Security Meeting in Pasay city, southeast of Manila, Philippines, in June 2017. [THE ASSOCIATED PRESS]',
 'th': '(ซ้ายสุด) นายติโต คาร์นาเวียน ผู้บัญชาการตํารวจแห่งชาติอินโดนีเซีย (จากซ้าย) นายโรนัลด์ เดลา โรซา ผู้บัญชาการตํารวจแห่งชาติฟิลิปปินส์ และนายคาลิด อาบู บาการ์ ผู้บัญชาการตํารวจแห่งชาติมาเลเซีย ไขว้แขนกันก่อนเริ่มการประชุมความมั่นคงไตรภาคีในเมืองปาเซย์ ซึ่งอยู่ทางตะวันออกเฉียงใต้ของกรุงมะนิลา ประเทศฟิลิปปินส์ ในเดือนมิถุนายน พ.ศ. 2560 ดิแอสโซซิเอทเต็ด เพรส'}

In [10]:
cut_datasets = DatasetDict()
for k in (raw_datasets.keys()):
    if k == 'train':
        cut_datasets[k] = Dataset.from_dict(raw_datasets[k][slice(0, 50000, None)])
    else:
        cut_datasets[k] = Dataset.from_dict(raw_datasets[k][slice(0, 1000, None)])

cut_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 1000
    })
})

In [11]:
import datasets, random
import pandas as pd
from IPython.display import display, HTML

def show_random_element(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than dataset"
    picks = []
    for n_ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks: # random inside
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [12]:
show_random_element(raw_datasets['train'])

#### Metric

In [13]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments).
    references: A list of one or more reference streams (each a sequence of segments).
    smooth_method: The smoothing method to use. (Default: 'exp').
    smooth_value: The smoothing value. Only valid for 'floor' and 'add-k'. (Defaults: floor: 0.1, add-k: 1).
    tokenize: Tokenization method to use for BLEU. If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for
        Japanese and '13a' (mteval) otherwise.
    lowercase: Lowercase the data. If True, enables case-insensitivity. (Default: False).
    force: Insist that your tokenized input is actually detokenized.

Returns:
    'score': BLEU score,
    'counts'

In [14]:
metric = load_metric("sacrebleu")fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## 1.2 Preprocessing the Data

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [16]:
# Set Source and Target Language

if 'mbart' in model_checkpoint:
    tokenizer.src_lang = 'th'
    tokenizer.tgt_lang = 'en'

In [17]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [70]:
max_input_len  = 128
max_target_len = 128

source_lang = 'th'
target_lang = 'en'

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    targets= [ex[target_lang] for ex in examples['translation']]
    
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_len, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [71]:
tokenized_datasets = cut_datasets.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [72]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['translation', 'subdataset', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['translation', 'subdataset', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

## 1.3 Fine-tuning the model

In [73]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-th-en/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0eb33d380c174e6420413d055247f46d08f0270c5b21934639530ea9cccdd756.2812763906fad9e04edf3afcd7e6da1e82df7c1856291e12132b346b18f9ee46
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-th-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62306
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62306,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "

In [81]:
batch_size = 20
model_name = model_checkpoint.split('/')[-1]
args = Seq2SeqTrainingArguments(
    f'{model_name}-finetuned-5k-{source_lang}-to-{target_lang}',
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    weight_decay =0.01,
    save_total_limit = 3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub = True,
)

TypeError: __init__() got an unexpected keyword argument 'max_split_size_mb'

#### Create Data collator

In [76]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [155]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels= [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    print('preds', preds)
    print('labels', labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [80]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

TypeError: __init__() got an unexpected keyword argument 'max_split_size_mb'

## 1.4 *train time!*

In [79]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation, subdataset. If translation, subdataset are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50000
  Num Epochs = 1
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 1
  Total optimization steps = 625
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead un

Saving model checkpoint to opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500
Configuration saved in opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500/config.json
Model weights saved in opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500/special_tokens_map.json
tokenizer config file saved in opus-mt-th-en-finetuned-5k-th-to-en/tokenizer_config.json
Special tokens file saved in opus-mt-th-en-finetuned-5k-th-to-en/special_tokens_map.json
Several commits (2) will be pushed upstream.
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation 

RuntimeError: CUDA out of memory. Tried to allocate 254.00 MiB (GPU 0; 15.78 GiB total capacity; 11.54 GiB already allocated; 165.69 MiB free; 13.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [85]:
?trainer.push_to_hub

Signature:
trainer.push_to_hub(
    commit_message: Union[str, NoneType] = 'End of training',
    blocking: bool = True,
    **kwargs,
) -> str
Docstring:
Upload *self.model* and *self.tokenizer* to the 🤗 model hub on the repo *self.args.hub_model_id*.

Parameters:
    commit_message (`str`, *optional*, defaults to `"End of training"`):
        Message to commit while pushing.
    blocking (`bool`, *optional*, defaults to `True`):
        Whether the function should return only when the `git push` has finished.
    kwargs:
        Additional keyword arguments passed along to [`~Trainer.create_model_card`].

Returns:
    The url of the commit of your model in the given repository if `blocking=False`, a tuple with the url of
    the commit and an object to track the progress of the commit if `blocking=True`
File:      /usr/local/lib/python3.8/dist-packages/transformers/trainer.py
Type:      method


In [176]:
!git lfs install
trainer.push_to_hub()

Git LFS initialized.


Saving model checkpoint to opus-mt-th-en-finetuned-5k-th-to-en
Configuration saved in opus-mt-th-en-finetuned-5k-th-to-en/config.json
Model weights saved in opus-mt-th-en-finetuned-5k-th-to-en/pytorch_model.bin
tokenizer config file saved in opus-mt-th-en-finetuned-5k-th-to-en/tokenizer_config.json
Special tokens file saved in opus-mt-th-en-finetuned-5k-th-to-en/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}


When Loading the model


```
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```



# 2. Using Translation Model

### Using AutoModel (With model selection)

In [93]:
?AutoModelForSeq2SeqLM.

Object `AutoModelForSeq2SeqLM.` not found.


In [162]:
tokenized_datasets['test']['labels'][0]

[987, 44, 188, 26, 1628, 93, 508, 2, 0]

In [ ]:
for model_input, gold_references in evaluation_dataset:
    model_predictions = model(model_inputs)
    metric.add_batch(predictions=model_predictions, references=gold_references)


In [153]:
# metric = load_metric("bleu")
# fake_preds = [["hello"],["there"], ["general"], ['kenobi']]
# fake_labels = [["hello"],["there"], ["general"], ['kenobi']]
# # fake_labels = [["hello there"], ["general kenobi"]]
# metric.compute(predictions=fake_preds, 
#                references=fake_labels)

In [183]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_pretrain = "Helsinki-NLP/opus-mt-th-en"
# model_for_predict = "./opus-mt-th-en-finetuned-5k-th-to-en"
model_for_predict = "huak95/opus-mt-th-en-finetuned-5k-th-to-en"

model_pt = AutoModelForSeq2SeqLM.from_pretrained(model_pretrain)
model = AutoModelForSeq2SeqLM.from_pretrained(model_for_predict)
tokenizer = AutoTokenizer.from_pretrained(model_pretrain)

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-th-en/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0eb33d380c174e6420413d055247f46d08f0270c5b21934639530ea9cccdd756.2812763906fad9e04edf3afcd7e6da1e82df7c1856291e12132b346b18f9ee46
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-th-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62306
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62306,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "

In [ ]:
!git lfs track "opus-mt-th-en-finetuned-5k-th-to-en/*"

model.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')
# tokenizer.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')

In [185]:
def harry_translate(input_texts, model=model_pt):
    inputs = tokenizer(input_texts['th'], return_tensors = "pt")

    outputs = model.generate(inputs["input_ids"],
                             max_length=40,
                             num_beams=4,
                             early_stopping=True)

    return tokenizer.decode(outputs[0]).replace('<pad> ',''), input_texts['en']

In [187]:
for i in range(10):
    text_ = tokenized_datasets['test']['translation'][i]
    thai_text = text_['th']
    pred_text_pt, real_text = harry_translate(text_, model_pt)
    pred_text, real_text = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text:',  thai_text)
    print('real_text: ', real_text)
    print('pred_text: ', pred_text)
    print('pre_train: ', pred_text_pt)
    print()

thai_text: ของพวกนี้ดีต่อการใช้งานตอนกลางคืน
real_text:  These are good for using at night.
pred_text:  These are good for night use.
pre_train:  They're good for night use.

thai_text: แม็กซ์ สมิธค่ะ
real_text:  It's Max Smith.
pred_text:  Max Smith.
pre_train:  Max Smith.

thai_text: ช่วยเอาเสื้อโค๊ตห้องแลปของเธอมาทีนะวันนี้ มิสคอลหาฉันหรือข้อความเมื่อเธออกนะ
real_text:  please bring your labcoat today.give me a misscall or message when u leave
pred_text:  Please take her lab coat today. Did Miss call me or a text message when she did?
pre_train:  Can you get me your lab coat today? Did Miss call me or her text when she did?

thai_text: ม็อบเปียกซับน้ําดีเยี่ยม สก๊อตช์-ไบรต์ XN002033480
real_text:  Scotch-Brite XN002033480 Wet Mop
pred_text:  The mob is a great water absorber, Scottish-bright XN002033480.
pre_train:  The mob's got a great sub, scotch-bright XN002033480.

thai_text: 500 หน่วยสําหรับคูปองสําหรับการสั่งซื้อ joybuy ใด ๆ!
real_text:  Coupon Detail 500 units for coupons fo